# qTESLA
O qTESLA é uma família de esquemas de assinatura digital lattice-based que fundamenta a sua segurança no problema de decisão do Ring Leaning With Errors (**R-LWE**).
A implementação que se segue assume os parâmetros recomendados do **qTESLA-p-I**.

## Parâmetros:
- o primo $p = 343576577$
- uma potencia de $2$, nomeadamente $n =1024$ 
- $d = 22$, sendo este o limite às truncaturas
- $B = 2^{19}-1$, corresponde ao limite para o polinómio aleatório na assinatura
- $L_e = 554$ (constante vinculada para o polinómio erro)
- $L_s = 554$ (constante vinculada para o polinómio secreto)
- $\sigma = 8.5$, que corresponde ao desvio padrão da distribuição gaussiana 
- $h = 25$ que é o parâmetro usado para a função de hash H

## KeyGen:
- Gerar polinómios $e,s$ segundo a distribuição gaussiana com desvio padrão $\sigma$
- Calcular a soma dos $h$ maiores elementos em valor absoluto de $e,s$ e se for maior que $L_e,L_s$ respetivamente, voltar ao passo inicial
- Gerar $a$ em $R_q$ invertível
- Calcular $t = a \cdot s + e \in \mathcal{R}_{q}$
- A chave pública é $(a,t) \in \mathcal{R}_{q}^{2}$
- A chave privada é $(s,e) \in \mathcal{R}^{2}$

## Sign: 
- Gerar aleatoriamente $r \in \mathcal{R}_{q}$ tal que a norma infinita de $r$ seja menor que $B$
- calcular $c = H([a \cdot r ]_M || m) \in \mathcal{R}_{q}$
- calcular $z = r + s \cdot c \in \mathcal{R}_q$
- se a norma infinita de $z$ + $L_s \geq B$ , voltar ao início
- se $a \cdot r - e \cdot c$ não for $L_e$ well-rounded, voltar ao início

A assinatura é o par $(z,c)$

## Verify:
Considerando a assinatura $(z,c)$ e a mensagem $m$:
- se a norma infinita de $z + L_s \geq B$ , rejeita-se a assinatura
- $w = a \cdot z - t \cdot c \in \mathcal{R}_q$
- se $ c \neq H([w]_{M} || m)$, rejeita-se a assinatura, caso contrário, a assinatura é aceite

In [15]:
from sage.stats.distributions.discrete_gaussian_polynomial import DiscreteGaussianDistributionPolynomialSampler

class qTesla:
    def __init__(self):
        self.q = 343576577
        self.n = 1024
        self.d = 22
        self.B = 2^19 - 1
        self.Le = 554
        self.Ls = 554
        self.sigma = 8.5
        self.h = 25
        self.alfa = self.sigma/self.q


    #FUNCOES AUXILIARES:

    # Least significant bits
    def Lsb(self,val):
        R = IntegerModRing(self.q)
        x = mod(R(val),2^self.d)
        return int(x)

    # Most significant bits
    def Msb(self,val):
        R = IntegerModRing(self.q)
        x = self.Lsb(val)
        y = (R(val) - x)/2^self.d
        return int(y)

    # Funcao Hash
    def Hash(self,val):
        H = []
        cont = 0
        cont_num = 0
        for i in range(0,self.n,2):
            u = val[i] + val[i+1]
            cont += 1
            if u == '11':
                H.append(0)
            if u == '01':
                H.append(1)
                cont_num += 1
            if u == '00':
                H.append(0)
            if u == '10':
                H.append(-1)
                cont_num += 1
            if cont_num > self.h or cont_num < self.h:
                break
        for i in range(self.n - 1):
            H.append(0)
        return H

    # Funcao auxiliar da funcao infinite_norm
    def pol_aux(self,w,n):
        Zx.<x> = ZZ[]
        r = int((n-1)//2)
        return Zx(map(lambda x: lift(x + r) - r , w.list()))

    # Norma infinita
    def infinite_norm(self,p,n):
        #R = self.pol_aux(p,n)
        J = p.list()
        for i in range(len(J)):
            J[i] = abs(int(J[i]))
        return int(max(J))

    # Funcao que verifica se o polinomio p e T-well-rounded
    def well_rounded(self,T,p):
        rounded = False
        norm = self.infinite_norm(p,self.q)
        lista = []
        u = p.list()
        for i in range(len(u)):
            k = self.Lsb(u[i])
            u[i] = abs(int(k))
        normL = max(u)
        if norm <= floor(self.q/2)-T and normL <= 2^(self.d-1) - T:
            rounded = True
        return rounded

    # FUNCOES PRINCIPAIS


    def Keygen(self):
        Zx.<x> = ZZ[]
        R.<x> = Zx.quotient(x^self.n+1)
        Zq.<z> = PolynomialRing(GF(self.q))
        Rq.<z> = Zq.quotient(z^self.n+1)

        # PASSO 1
        e = DiscreteGaussianDistributionPolynomialSampler(R, self.n, self.sigma)()
        self.eq = DiscreteGaussianDistributionPolynomialSampler(Rq, self.n, self.sigma)()
        s = DiscreteGaussianDistributionPolynomialSampler(R, self.n, self.sigma)()
        self.sq = DiscreteGaussianDistributionPolynomialSampler(Rq, self.n, self.sigma)()

        # PASSO 2
        E = e.list()
        for i in range(len(E)):
            E[i] = abs(int(E[i]))
        S = s.list()
        for i in range(len(S)):
            S[i] = abs(int(S[i]))
        E.sort()
        S.sort()
        E = E[self.n-self.h:]
        S = S[self.n-self.h:]
        while sum(E) >= self.Le or sum(S) >= self.Ls:
            e = DiscreteGaussianDistributionPolynomialSampler(R, self.n, self.sigma)()
            self.eq = DiscreteGaussianDistributionPolynomialSampler(Rq, self.n, self.sigma)()
            s = DiscreteGaussianDistributionPolynomialSampler(R, self.n, self.sigma)()
            E = e.list()
            for i in range(len(E)):
                E[i] = abs(int(E[i]))
            S = s.list()
            for i in range(len(S)):
                S[i] = abs(int(S[i]))
            E.sort()
            S.sort()
            E = E[self.n - self.d:]
            S = S[self.n - self.d:]

        # PASSO 3
        a = Rq.random_element()
        while not a.is_unit():
            a = Rq.random_element()
        self.aq = a

        t = self.aq*self.sq + self.eq

        # PASSO 4 E 5
        self.PubKey = (a,t)
        self.PrivKey = (s,e)
        return(self.PubKey,self.PrivKey)


    def Sign(self,m):
        Zx.<x> = ZZ[]
        R.<x> = Zx.quotient(x^self.n+1)
        Zq.<z> = PolynomialRing(GF(self.q))
        Rq.<z> = Zq.quotient(z^self.n+1)

        # PASSO 1
        r = []
        for i in range(self.n):
            r.append(randint(1,self.B))
        rq = Rq(r)
        r = R(r)

        # PASSO 2
        a = self.PubKey[0]
        u = self.aq*rq
        u = u.list()
        string = ""
        l = m[2:]
        string = string + l
        for i in range(self.n):
            u[i] = self.Msb(u[i])
            l = bin(u[i])
            if u[i] >= 0:
                string = string + l[2:]
            if u[i] < 0:
                string = string + l[3:]
        c = self.Hash(string)
        cq = Rq(c)
        cR = R(c)

        # PASSO 3
        s = self.PrivKey[0]
        e = self.PrivKey[1]

        z = rq+self.sq*cq

        # PASSO 4, 5 E 6
        wb = self.aq*rq - self.eq*cq
        while self.infinite_norm(z,self.q) + self.Ls >= self.B and self.well_rounded(self.Le,wb) == False:
            r = []
            for i in range(self.n):
                r.append(randint(1,self.B))
            rq = Rq(r)
            r = R(r)
            a = self.PubKey[0]
            u = self.aq*rq
            u = u.list()
            string = ''
            l = m[2:]
            string = string + l
            for i in range(self.n):
                u[i] = self.Msb(u[i])
                l = bin(u[i])
                if u[i] >= 0:
                    string = string + l[2:]
                if u[i] < 0:
                    string = string + l[3:]
            c = self.Hash(string)
            cq = Rq(c)
            cR = R(c)
            s = self.PrivKey[0]
            e = self.PrivKey[1]
            z = rq + self.sq*cq
            wb = self.aq*rq-self.eq*cq
        sign = (z,c)
        return sign



    def Verify(self,m,sign):
        z = sign[0]
        c = sign[1]
        a = self.PubKey[0]
        t = self.PubKey[1]


        Zx.<x> = ZZ[]
        R.<x> = Zx.quotient(x^self.n+1)
        Zq.<z> = PolynomialRing(GF(self.q))
        Rq.<z> = Zq.quotient(z^self.n+1)

        # PASSO 1
        if self.infinite_norm(z,self.q) + self.Ls >= self.B:
            print ("Assinatura rejeitada!")

        # PASSO 2
        cq = Rq(c)
        w = self.aq*z - t*cq

        # PASSO 3 E 4
        u = w.list()
        string = ""
        l = m[2:]
        string = string + l
        for i in range(self.n):
            u[i] = self.Msb(u[i])
            k = bin(u[i])
            if u[i] >= 0:
                string = string + l[2:]
            if u[i] < 0:
                string = string + l[3:]
        Hash = self.Hash(string)
        if c != Hash:
            print ("Assinatura rejeitada!")
        else:
            print ("Assinatura aceite!")

In [16]:
Teste = qTesla()
chaves = Teste.Keygen()
m = bin(123456789)
sign = Teste.Sign(m)
print('Mensagem: ',m)
print('Par de chaves: ', c)
#print('Assinatura: ', sign)
Teste.Verify(m,sign)

Mensagem:  0b111010110111100110100010101
Assinatura aceite!


In [3]:
for i in range (1,100):
    Teste = qTesla()
    Teste.Keygen()
    m = bin(randint(1,99999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999))
    sign = Teste.Sign(m)
    Teste.Verify(m,sign)

Assinatura aceite!
Assinatura aceite!


Assinatura aceite!
Assinatura aceite!


Assinatura aceite!
Assinatura aceite!


Assinatura aceite!


Assinatura aceite!
Assinatura aceite!


...

Assinatura aceite!
Assinatura aceite!


Assinatura aceite!


Assinatura aceite!
Assinatura aceite!
Assinatura aceite!


Assinatura aceite!
Assinatura aceite!


Assinatura aceite!


# Zita Abreu